In [1]:
import qnmfits
import numpy as np
import scipy
import corner
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import sys
from pathlib import Path

notebook_dir = Path().resolve()
sys.path.append(str(notebook_dir.parent))

notebook_dir = Path().resolve()
data_dir = notebook_dir.parent

from matplotlib.colors import to_hex
from matplotlib.lines import Line2D
from matplotlib.colors import LinearSegmentedColormap
from qnmfits.spatial_mapping_functions import * 
from bayes_qnm_GP_likelihood import *
from BGP_fits import BGP_fit
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

In [2]:
id = "0001"
sim_main = SXS_CCE(id, lev="Lev5", radius="R2")

n_max = 7

T0 = 0
T = 100

qnm_list = [(2,2,n,1) for n in np.arange(0, n_max+1)] #+ [(3,2,0,1)] + [(4,4,n,1) for n in np.arange(0, n_max+1)] + [(5,4,0,1)] 
spherical_modes = [(2, 2)] 

chif_mag_0 = sim_main.chif_mag
Mf_0 = sim_main.Mf

In [3]:
# Get the standard kernel parameters

with open(data_dir / 'bayes_qnm_GP_likelihood/data/tuned_params.pkl', 'rb') as f:
    params = pickle.load(f)

tuned_param_dict_main = params[id]

In [4]:
# The training / tuning is demonstrated in the `get_kernel_params` file and notebook. 

with open(data_dir / 'bayes_qnm_GP_likelihood/data/param_dict_sim_lm_full.pkl', 'rb') as f:
    param_dict_sim_lm = pickle.load(f)

tuning_hyperparams_s = [0.3239257812499994]  

hyperparam_rule_dict_s = {
    "sigma_max": "multiply",
}

tuned_param_dict_wn = {mode: get_new_params(param_dict_sim_lm[id][mode], tuning_hyperparams_s, hyperparam_rule_dict_s) for mode in param_dict_sim_lm[id]}

In [5]:
fit = BGP_fit(sim_main.times, 
            sim_main.h, 
            qnm_list, 
            Mf_0, 
            chif_mag_0, 
            T0, 
            tuned_param_dict_wn, 
            kernel_s, 
            t0_method='closest', 
            T=T, 
            spherical_modes=spherical_modes,
            include_chif=True,
            include_Mf=True)

Time taken for parameter Re_C_(2, 2, 0, 1): 0.07766342163085938 seconds
Time taken for parameter Re_C_(2, 2, 1, 1): 0.0014386177062988281 seconds
Time taken for parameter Re_C_(2, 2, 2, 1): 0.001069784164428711 seconds
Time taken for parameter Re_C_(2, 2, 3, 1): 0.0010306835174560547 seconds
Time taken for parameter Re_C_(2, 2, 4, 1): 0.0010154247283935547 seconds
Time taken for parameter Re_C_(2, 2, 5, 1): 0.001003265380859375 seconds
Time taken for parameter Re_C_(2, 2, 6, 1): 0.0009944438934326172 seconds
Time taken for parameter Re_C_(2, 2, 7, 1): 0.000997304916381836 seconds
Time taken for parameter chif: 0.08174800872802734 seconds
Time taken for parameter Mf: 0.05586719512939453 seconds


In [9]:
print(fit.mean_vector)

[  0.96211353  -0.19918416  -2.97059033  -2.98815029   4.2896938
  10.55836519  -8.91749015 -21.74737427  20.0696907   28.38812926
 -25.22777948 -19.98126399  14.95535981   6.4482694   -3.37579294
  -0.81059088   0.68736206   0.95208702]


In [10]:
fit_main = qnm_BGP_fit(
        sim_main.times,
        sim_main.h,
        qnm_list,
        Mf_0,
        chif_mag_0,
        T0,
        tuned_param_dict_wn, 
        kernel_s, 
        t0_method="closest",
        T=T,
        spherical_modes=spherical_modes,
        include_chif=True,
        include_Mf=True,
    )

In [ ]:
print(fit_main['mean_vector'])